In [1]:
%cd ..

/Users/ishamrashik/Desktop/machine-learning-and-ai-projects/community-mcp-session/aws_community_day/youtube-sentiment-insights


In [2]:
import mlflow
import os 

# Set AWS credentials (replace with your actual credentials)
os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('AWS_SECRET_ACCESS_KEY')
os.environ['AWS_DEFAULT_REGION'] = os.getenv('AWS_DEFAULT_REGION')

# Then set MLflow tracking URI
mlflow.set_tracking_uri("http://3.29.137.20:5000")

In [3]:
# Set or create an experiment
mlflow.set_experiment("Exp 3 - TfIdf Trigram max_features")

2025/10/12 10:53:31 INFO mlflow.tracking.fluent: Experiment with name 'Exp 3 - TfIdf Trigram max_features' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-artifacts-5fb77f1c/466738871655413067', creation_time=1760252011577, experiment_id='466738871655413067', last_update_time=1760252011577, lifecycle_stage='active', name='Exp 3 - TfIdf Trigram max_features', tags={}>

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [10]:
from utilities import load_data, INTERIM_DATA_PATH

In [11]:
train_df_path = os.path.join(INTERIM_DATA_PATH, "train_processed.csv")
val_df_path = os.path.join(INTERIM_DATA_PATH, "val_processed.csv")
test_df_path = os.path.join(INTERIM_DATA_PATH, "test_processed.csv")

train_df = load_data(train_df_path)
val_df = load_data(val_df_path)
test_df = load_data(test_df_path)

2025-10-12 11:42:47,522 - model_building - DEBUG - Data loaded from data/interim/train_processed.csv
2025-10-12 11:42:47,525 - model_building - DEBUG - Data loaded from data/interim/val_processed.csv
2025-10-12 11:42:47,530 - model_building - DEBUG - Data loaded from data/interim/test_processed.csv


In [ ]:
# Step 1: Function to run the experiment
def run_experiment_tfidf_max_features(max_features):
    ngram_range = (1, 1)  # BoW setting

    # Step 2: Vectorization using TF-IDF with varying max_features
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

    # Step 1: Vectorize the text comments (Bag of Words)
    X_train_text = vectorizer.fit_transform(train_df['clean_comment']).toarray()
    X_val_text = vectorizer.transform(val_df['clean_comment']).toarray()

    # Step 2: Extract numerical features
    # Define the numerical feature columns to include
    numerical_features = ['word_count', 'num_stop_words', 'num_chars', 'num_chars_cleaned']

    X_train_numerical = train_df[numerical_features].values
    X_val_numerical = val_df[numerical_features].values

    # Step 3: Combine text features with numerical features
    import numpy as np

    X_train = np.hstack([X_train_text, X_train_numerical])
    X_val = np.hstack([X_val_text, X_val_numerical])

    # Target variable
    y_train = train_df['category']
    y_val = val_df['category']


    # Step 4: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"TFIDF_Trigrams_max_features_{max_features}")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with TF-IDF Trigrams, max_features={max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_val)

        # Log accuracy
        accuracy = accuracy_score(y_val, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_val, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_val, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Trigrams, max_features={max_features}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_tfidf_trigrams_{max_features}")

# Step 6: Test various max_features values
max_features_values = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for max_features in max_features_values:
    run_experiment_tfidf_max_features(max_features)

2025/10/12 10:59:46 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/12 10:59:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigrams_max_features_1000 at: http://3.29.137.20:5000/#/experiments/466738871655413067/runs/d7296d82dc164f0fadfe0130452f296c
🧪 View experiment at: http://3.29.137.20:5000/#/experiments/466738871655413067


/opt/homebrew/Caskroom/miniforge/base/envs/mlops_aws_community/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/homebrew/Caskroom/miniforge/base/envs/mlops_aws_community/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/homebrew/Caskroom/miniforge/base/envs/mlops_aws_community/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divi

🏃 View run TFIDF_Trigrams_max_features_2000 at: http://3.29.137.20:5000/#/experiments/466738871655413067/runs/2997208b6fab4b97ab5997c9d542706d
🧪 View experiment at: http://3.29.137.20:5000/#/experiments/466738871655413067


2025/10/12 11:00:04 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/12 11:00:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run TFIDF_Trigrams_max_features_8000 at: http://3.29.137.20:5000/#/experiments/466738871655413067/runs/994b5b1f7945426791ed7611705fad91
🧪 View experiment at: http://3.29.137.20:5000/#/experiments/466738871655413067


/opt/homebrew/Caskroom/miniforge/base/envs/mlops_aws_community/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/homebrew/Caskroom/miniforge/base/envs/mlops_aws_community/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/homebrew/Caskroom/miniforge/base/envs/mlops_aws_community/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divi

🏃 View run TFIDF_Trigrams_max_features_9000 at: http://3.29.137.20:5000/#/experiments/466738871655413067/runs/b4b9b04d3104439ebc4bb46792efd93f
🧪 View experiment at: http://3.29.137.20:5000/#/experiments/466738871655413067


/opt/homebrew/Caskroom/miniforge/base/envs/mlops_aws_community/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/homebrew/Caskroom/miniforge/base/envs/mlops_aws_community/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/homebrew/Caskroom/miniforge/base/envs/mlops_aws_community/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divi

🏃 View run TFIDF_Trigrams_max_features_10000 at: http://3.29.137.20:5000/#/experiments/466738871655413067/runs/c0cd7b2b168a40acaa5a5f1eed04a2ea
🧪 View experiment at: http://3.29.137.20:5000/#/experiments/466738871655413067
